In [4]:
import yfinance as yahooFinance
import pandas as pd
import numpy as np
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import norm 
import matplotlib.pyplot as plt
import pandas_datareader.data as web

In [5]:
ticker_symbol = "^GSPC"
start_date = "2021-01-31"
end_date = "2022-12-31"
sp500_data = yahooFinance.download(ticker_symbol, start=start_date,
                                   end=end_date, interval="1d")
# sp500_data.to_excel('data.xlsx')AV

[*********************100%%**********************]  1 of 1 completed


In [7]:
#  retrieve data from the Federal Reserve Economic Data (FRED) database. 
# fetches data for the 3-Month Treasury Bill (DTB3) from FRED.
X2 = web.DataReader('DTB3','fred',start_date,end_date)
X2

,DTB3
DATE,
2021-02-01,0.07
2021-02-02,0.07
2021-02-03,0.04
2021-02-04,0.04
2021-02-05,0.03
...,...
2022-12-26,NaN
2022-12-27,4.35
2022-12-28,4.35


In [9]:
sp500_data['Adjusted_Return'] = sp500_data['Adj Close'].pct_change()
sp500_data.drop(['Open','High','Low','Close','Volume'],
                inplace = True, axis = 1)

In [10]:
# Group by month and select the first day of each month
first_day_of_month = sp500_data.groupby(pd.Grouper(freq='MS')).first()

In [13]:
sp500_data['rolling_vol'] = sp500_data[
    'Adjusted_Return'].rolling(window=10).std()
sp500_data.dropna(inplace = True)
sp500_data

,Adj Close,Adjusted_Return,rolling_vol
Date,,,
2021-03-12,3943.340088,0.001015,0.013393
2021-03-15,3968.939941,0.006492,0.011458
2021-03-16,3962.709961,-0.001570,0.011013
2021-03-17,3974.120117,0.002879,0.009584
2021-03-18,3915.459961,-0.014761,0.009860
...,...,...,...
2022-12-23,3844.820068,0.005868,0.013064
2022-12-27,3829.250000,-0.004050,0.011706
2022-12-28,3783.219971,-0.012021,0.011209


In [14]:
sp500_data = pd.merge(sp500_data,X2, left_index = True,
                      right_index = True, how = 'inner')
cols = sp500_data.columns.to_list()
cols[-1] = 'Rf'
sp500_data.columns = cols
sp500_data

,Adj Close,Adjusted_Return,rolling_vol,Rf
2021-03-12,3943.340088,0.001015,0.013393,0.04
2021-03-15,3968.939941,0.006492,0.011458,0.04
2021-03-16,3962.709961,-0.001570,0.011013,0.02
2021-03-17,3974.120117,0.002879,0.009584,0.02
2021-03-18,3915.459961,-0.014761,0.009860,0.01
...,...,...,...,...
2022-12-23,3844.820068,0.005868,0.013064,4.23
2022-12-27,3829.250000,-0.004050,0.011706,4.35
2022-12-28,3783.219971,-0.012021,0.011209,4.35
2022-12-29,3849.280029,0.017461,0.013438,4.34


In [8]:
max(sp500_data['Adj Close'])

4796.56005859375

In [16]:
possible_strike = list(range(2300, 5400 + 1, 25))
possible_strike = np.array(possible_strike)
possible_strike

array([2300, 2325, 2350, 2375, 2400, 2425, 2450, 2475, 2500, 2525, 2550,
       2575, 2600, 2625, 2650, 2675, 2700, 2725, 2750, 2775, 2800, 2825,
       2850, 2875, 2900, 2925, 2950, 2975, 3000, 3025, 3050, 3075, 3100,
       3125, 3150, 3175, 3200, 3225, 3250, 3275, 3300, 3325, 3350, 3375,
       3400, 3425, 3450, 3475, 3500, 3525, 3550, 3575, 3600, 3625, 3650,
       3675, 3700, 3725, 3750, 3775, 3800, 3825, 3850, 3875, 3900, 3925,
       3950, 3975, 4000, 4025, 4050, 4075, 4100, 4125, 4150, 4175, 4200,
       4225, 4250, 4275, 4300, 4325, 4350, 4375, 4400, 4425, 4450, 4475,
       4500, 4525, 4550, 4575, 4600, 4625, 4650, 4675, 4700, 4725, 4750,
       4775, 4800, 4825, 4850, 4875, 4900, 4925, 4950, 4975, 5000, 5025,
       5050, 5075, 5100, 5125, 5150, 5175, 5200, 5225, 5250, 5275, 5300,
       5325, 5350, 5375, 5400])

In [17]:
sp500_data['Time To Maturity'] = 1/12
sp500_data['Price Option'] = 0
sp500_data.drop('Adjusted_Return', axis = 1, inplace = True)
# Group by month and select the first day of each month
sp500_data = sp500_data.groupby(pd.Grouper(freq='MS')).first()
sp500_data

,Adj Close,rolling_vol,Rf,Time To Maturity,Price Option
2021-03-01,3943.340088,0.013393,0.04,0.083333,0
2021-04-01,4019.870117,0.007708,0.02,0.083333,0
2021-05-01,4192.660156,0.006999,0.04,0.083333,0
2021-06-01,4202.040039,0.005704,0.02,0.083333,0
2021-07-01,4319.939941,0.006841,0.05,0.083333,0
2021-08-01,4387.160156,0.006608,0.05,0.083333,0
2021-09-01,4524.089844,0.004736,0.05,0.083333,0
2021-10-01,4357.040039,0.011546,0.04,0.083333,0
2021-11-01,4613.669922,0.004142,0.05,0.083333,0
2021-12-01,4513.040039,0.010888,0.06,0.083333,0


In [18]:
# Define the dimensions of your 4D array
num_call_or_put = ["Call","Put"]  # Call and Put
num_strike_prices = possible_strike
# date_index = date_arr
# num_rows_dates = len(date_arr)

for j,val in enumerate(num_strike_prices):
    sp500_data['Strike'] = np.array([val]*sp500_data.shape[0])
    sp500_data['Profit/Loss Options'] = np.array([0]*sp500_data.shape[0])
    sp500_data['Profit/Loss Spot'] = np.array([0]*sp500_data.shape[0])

ind = sp500_data.index
col = sp500_data.columns
    
num_rows, num_columns = sp500_data.shape    
    
# Create a 4D array to store the data
four_dimensional_array = np.zeros((len(num_call_or_put),
                                   len(num_strike_prices),
                                   num_rows, num_columns), dtype=object)

for i in range(len(num_call_or_put)):
    for j,val in enumerate(num_strike_prices):
        sp500_data['Strike'] = np.array([val]*num_rows)
        four_dimensional_array[i,j,:,:] = sp500_data

In [19]:
def calculate_pnl(df,option_type):
    if option_type == 'Call':
        df['Profit/Loss Options'] = -df['Price Option'].diff().fillna(0) * 100
    else:
        df['Profit/Loss Options'] = -df['Price Option'].diff().fillna(0) * 100
    df['Profit/Loss Spot'] = df['Adj Close'].diff().fillna(0) * 100
    return df

def blackscholes(df, option_type): 
    S_t = df['Adj Close']
    K = df['Strike']
    r = df['Rf']
    sigma = df['rolling_vol']
    T_t = df['Time To Maturity']
    def d1(S_t, K, r, sigma, T_t): 
        return (np.log(S_t / K) + (r + 0.5 * sigma**2) * T_t) / (sigma * np.sqrt(T_t)) 
    
    def d2(d1, sigma, T_t): 
        return d1 - sigma * np.sqrt(T_t) 
    
    # Calculate d1, d2, and the Black-Scholes option price for each row in the dataframe 
    if option_type == 'Call':
        df['d1'] = d1(S_t, K, r, sigma, T_t) 
        df['d2'] = d2(df['d1'], sigma, T_t) 
        df['Price Option'] = S_t * norm.cdf(df['d1']) - K * np.exp(
            -r * T_t) * norm.cdf(df['d2'])
        df['Delta'] = norm.cdf(df['d1'])
    elif option_type == 'Put':
        df['d1'] = d1(S_t, K, r, sigma, T_t) 
        df['d2'] = d2(df['d1'], sigma, T_t) 
        df['Price Option'] = K * np.exp(-r * T_t) * norm.cdf(
            -df['d2']) - S_t * norm.cdf(-df['d1'])
        df['Delta'] = -norm.cdf(-df['d1'])
    else:
        raise ValueError("Invalid option_type. Use 'call' or 'put'.")
    
#   calulting pnl from options price difference on day to day basis
    df = calculate_pnl(df,option_type)
    # Drop d1 and d2 columns 
    df = df.drop(['d1', 'd2'], axis=1) 
    return df

In [21]:
option_type = ('Call', 'Put')
# possible_strikes

for i, opt in enumerate(option_type):
    # Create an Excel writer object
    excel_writer = pd.ExcelWriter(opt + '.xlsx', engine='xlsxwriter')
    for j, val in enumerate(possible_strike):
        df = pd.DataFrame(four_dimensional_array[i,j,:,:], index = ind,
                          columns = col)
        df = df.astype(float) 
        df = blackscholes(df, opt)
        df.to_excel(excel_writer, sheet_name = str(val), index=True)
    excel_writer.close()        
        



In [22]:
profit_loss_pf1 = pd.DataFrame(columns = ['Date','Call_Strike','Put_Strike','Profit/Loss'])
profit_loss_pf1['Date'] = sp500_data.index
profit_loss_pf1 = pd.merge(profit_loss_pf1,sp500_data['Adj Close'],
                           left_on='Date', right_index=True, how='inner')

profit_loss_pf1['Call_Strike'] = np.ceil(profit_loss_pf1['Adj Close'] * 1.03 / 25) * 25
profit_loss_pf1['Put_Strike'] = np.ceil(profit_loss_pf1['Adj Close'] * 1.05 / 25) * 25
# profit_loss_pf1

In [23]:
# profit_loss_pf1
excel_file_path = 'Call.xlsx'
arr = []
for row in profit_loss_pf1.iterrows():
    call_strike = str(int(row[1][1]))
    df = pd.read_excel(excel_file_path, sheet_name=call_strike)
    entry = df[df.index == row[0]]['Profit/Loss Options'].iloc[0]

    # Append the entry to arr
    arr.append(entry)
arr = pd.DataFrame({'Profit/Loss': arr}, index=profit_loss_pf1['Date'])
profit_loss_pf1 = pd.merge(profit_loss_pf1, arr, left_on='Date',
                           right_index=True, how='inner')

In [24]:
excel_file_path = 'Put.xlsx'
arr = []
for row in profit_loss_pf1.iterrows():
    call_strike = str(int(row[1][2]))
    df = pd.read_excel(excel_file_path, sheet_name=call_strike)
    entry = df[df.index == row[0]]['Profit/Loss Options'].iloc[0]

    # Append the entry to arr
    arr.append(entry)
arr = pd.DataFrame({'Profit/Loss Put': arr}, index=profit_loss_pf1['Date'])
profit_loss_pf1 = pd.merge(profit_loss_pf1, arr, left_on='Date',
                           right_index=True, how='inner')


In [25]:
excel_output_path = 'profit_loss_3pf1_output.xlsx'

# Save the DataFrame to Excel
profit_loss_pf1.to_excel(excel_output_path, index=False)

In [26]:
profit_loss_pf2 = pd.DataFrame(columns = ['Date','Call_Strike','Put_Strike','Profit/Loss'])
profit_loss_pf2['Date'] = sp500_data.index
profit_loss_pf2 = pd.merge(profit_loss_pf2,sp500_data['Adj Close'],
                           left_on='Date', right_index=True, how='inner')

profit_loss_pf2['Call_Strike'] = np.ceil(profit_loss_pf1[
    'Adj Close'] * 1.05 / 25) * 25
profit_loss_pf2['Put_Strike'] = np.ceil(profit_loss_pf1[
    'Adj Close'] * 1.03 / 25) * 25

excel_file_path = 'Call.xlsx'
arr = []
for row in profit_loss_pf1.iterrows():
    call_strike = str(int(row[1][1]))
    df = pd.read_excel(excel_file_path, sheet_name=call_strike)
    entry = df[df.index == row[0]]['Profit/Loss Options'].iloc[0]

    # Append the entry to arr
    arr.append(entry)
arr = pd.DataFrame({'Profit/Loss': arr}, index=profit_loss_pf1['Date'])
profit_loss_pf2 = pd.merge(profit_loss_pf2, arr, left_on='Date',
                           right_index=True, how='inner')



In [27]:
excel_file_path = 'Put.xlsx'
arr = []
for row in profit_loss_pf2.iterrows():
    call_strike = str(int(row[1][2]))
    df = pd.read_excel(excel_file_path, sheet_name=call_strike)
    entry = df[df.index == row[0]]['Profit/Loss Options'].iloc[0]

    # Append the entry to arr
    arr.append(entry)
arr = pd.DataFrame({'Profit/Loss Put': arr}, index=profit_loss_pf2['Date'])
profit_loss_pf2 = pd.merge(profit_loss_pf2, arr, left_on='Date',
                           right_index=True, how='inner')

In [28]:
excel_output_path = 'profit_loss_3pf2_output.xlsx'

# Save the DataFrame to Excel
profit_loss_pf2.to_excel(excel_output_path, index=False)

In [29]:
profit_loss_pf3 = pd.DataFrame(columns = ['Date','Call_Strike',
                                          'Put_Strike','Profit/Loss'])
profit_loss_pf3['Date'] = sp500_data.index
profit_loss_pf3 = pd.merge(profit_loss_pf3,sp500_data['Adj Close'],
                           left_on='Date', right_index=True, how='inner')

profit_loss_pf3['Call_Strike'] = np.ceil(profit_loss_pf3[
    'Adj Close'] * 1.03 / 25) * 25
profit_loss_pf3['Put_Strike'] = np.ceil(profit_loss_pf3[
    'Adj Close'] * 1.07 / 25) * 25

excel_file_path = 'Call.xlsx'
arr = []
for row in profit_loss_pf1.iterrows():
    call_strike = str(int(row[1][1]))
    df = pd.read_excel(excel_file_path, sheet_name=call_strike)
    entry = df[df.index == row[0]]['Profit/Loss Options'].iloc[0]

    # Append the entry to arr
    arr.append(entry)
arr = pd.DataFrame({'Profit/Loss': arr}, index=profit_loss_pf1['Date'])
profit_loss_pf3 = pd.merge(profit_loss_pf3, arr, left_on='Date',
                           right_index=True, how='inner')



In [30]:
# profit_loss_pf1
excel_file_path = 'Put.xlsx'
# df = pd.read_excel(excel_file_path, sheet_name='1350')
#     # Display the data
# print(df)
arr = []
for row in profit_loss_pf3.iterrows():
    call_strike = str(int(row[1][2]))
    df = pd.read_excel(excel_file_path, sheet_name=call_strike)
    entry = df[df.index == row[0]]['Profit/Loss Options'].iloc[0]

    # Append the entry to arr
    arr.append(entry)
arr = pd.DataFrame({'Profit/Loss Put': arr}, index=profit_loss_pf3['Date'])
profit_loss_pf3 = pd.merge(profit_loss_pf3, arr, left_on='Date',
                           right_index=True, how='inner')

In [31]:
excel_output_path = 'profit_loss_3pf3_output.xlsx'

# Save the DataFrame to Excel
profit_loss_pf3.to_excel(excel_output_path, index=False)

In [32]:
profit_loss_pf4 = pd.DataFrame(columns = ['Date','Call_Strike',
                                          'Put_Strike','Profit/Loss'])
profit_loss_pf4['Date'] = sp500_data.index
profit_loss_pf4 = pd.merge(profit_loss_pf4,sp500_data['Adj Close'],
                           left_on='Date', right_index=True, how='inner')

profit_loss_pf4['Call_Strike'] = np.ceil(profit_loss_pf4[
    'Adj Close'] * 1.05 / 25) * 25
profit_loss_pf4['Put_Strike'] = np.ceil(profit_loss_pf4[
    'Adj Close'] * 1.01 / 25) * 25

excel_file_path = 'Call.xlsx'
arr = []
for row in profit_loss_pf1.iterrows():
    call_strike = str(int(row[1][1]))
    df = pd.read_excel(excel_file_path, sheet_name=call_strike)
    entry = df[df.index == row[0]]['Profit/Loss Options'].iloc[0]

    # Append the entry to arr
    arr.append(entry)
arr = pd.DataFrame({'Profit/Loss': arr}, index=profit_loss_pf4['Date'])
profit_loss_pf4 = pd.merge(profit_loss_pf4, arr, left_on='Date',
                           right_index=True, how='inner')



In [33]:
# profit_loss_pf1
excel_file_path = 'Put.xlsx'
# df = pd.read_excel(excel_file_path, sheet_name='1350')
#     # Display the data
# print(df)
arr = []
for row in profit_loss_pf4.iterrows():
    call_strike = str(int(row[1][2]))
    df = pd.read_excel(excel_file_path, sheet_name=call_strike)
    entry = df[df.index == row[0]]['Profit/Loss Options'].iloc[0]

    # Append the entry to arr
    arr.append(entry)
arr = pd.DataFrame({'Profit/Loss Put': arr}, index=profit_loss_pf4['Date'])
profit_loss_pf4 = pd.merge(profit_loss_pf4, arr, left_on='Date',
                           right_index=True, how='inner')

excel_output_path = 'profit_loss_3pf4_output.xlsx'

# Save the DataFrame to Excel
profit_loss_pf4.to_excel(excel_output_path, index=False)